# init

## install openpyxl

pandas uses this when saving Excel files. pandas imports it automatically, as needed.

In [2]:
!pip install openpyxl

## import Python modules

In [3]:
import pandas, IPython.core.display

## open file

In [4]:
src = pandas.ExcelFile('2017 CAM data from iPads.xlsx')
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

## select sheets

In [83]:
search_strings = ['Erl', 'Tyler']
sheets_to_process = {
    sheet_name.split(' ')[-1]: src.parse(sheet_name)
    for sheet_name in src.sheet_names
    if any(
        (
            (pattern in sheet_name) for pattern in search_strings
        )
    )
}
sheets_to_process.keys()

dict_keys(['Erl', 'Tyler'])

## Define Aphids and Natural Enemies

In [88]:
EGA = 'English grain'
BCO = 'bird cherry oat'
greenbug = 'green bug'
aphid_name_list = (EGA, BCO, greenbug)
aphid_name = {f'a{n + 1}': name for n, name in enumerate(aphid_name_list)}
aphid_name

{'a1': 'English grain', 'a2': 'bird cherry oat', 'a3': 'green bug'}


In [84]:
# @todo: input natural enemy names
natural_enemy_name_list = [
    'aphid_mummies_brown',
    'aphid_mummies_blk',
    'ladybeetle_larvae',
    'ladybeetle_adult',
    'lacewing_larvae',
    'lacewing_adult',
]
natural_enemy_name = {f'e{n + 1}': name for n, name in enumerate(natural_enemy_name_list)}
natural_enemy_name

{'e1': 'aphid_mummies_brown',
 'e2': 'aphid_mummies_blk',
 'e3': 'ladybeetle_larvae',
 'e4': 'ladybeetle_adult',
 'e5': 'lacewing_larvae',
 'e6': 'lacewing_adult'}

# cleanup

## normalize column names

In [90]:
sheets_needing_column_rename = (
    (sheet_name, sheet)
    for sheet_name, sheet
    in sheets_to_process.items()
    if any(
        (str(column).find(' ')
         for column in sheet.columns)
    )
)
sheets_to_process = {
    sheet_name: sheet.rename(
        mapper=lambda x: str(x).split(' ')[0],
        axis='columns'
    )
    for sheet_name, sheet in sheets_needing_column_rename
}

## concatenate sheets

In [92]:
concat_df = pandas.concat(
    sheets_to_process,
    names=['worksheet_name', 'index']
)

## reindex and rename columns

In [225]:
names = ['scope', 'group', 'variable', 'source_field']
a1, a2, a3 = aphid_name
columns = (
    ('field',          '','client',                 'fields__client__displayText'),
    ('field',          '','crop',                   'fields__crop'),
    ('field',          '','date',                   'fields__date'),
    ('field',          '','desc',                   'fields__desc'),
    ('field',          '','name',                   'fields__name'),
    ('set',            '','date',                   'fields__oSets__date'),
    ('set',            '','description',            'fields__oSets__desc'),
    ('set',            '','growth_stage',           'fields__oSets__growthStage'),
    ('set',            '','observer',               'fields__oSets__obsName'),
    ('set',            '','complete_sets',          'fields__oSets__completeSets'),
    ('set',            '','total_sets',             'fields__oSets__totalSets'),
    ('point',          '','id',                     'fields__oSets__oPoints__id'),
    ('observation',    '','id',                     'fields__oSets__oPoints__observations__id'),
    ('observation',    '','complete',               'fields__oSets__oPoints__observations__complete'),
    ('observation',    '','disabled',               'fields__oSets__oPoints__observations__disabled'),
    ('observation',    '','natural_enemy_subtotal', 'fields__oSets__oPoints__observations__enum'),
    ('observation',    '','aphid_subtotal',         'fields__oSets__oPoints__observations__anum'),
    ('observation',    '','a1',                     'fields__oSets__oPoints__observations__a1__number'),
    ('observation',    '','a2',                     'fields__oSets__oPoints__observations__a2__number'),
    ('observation',    '','a3',                     'fields__oSets__oPoints__observations__a3__number'),
    ('observation',    'natural_enemy', 'name',  'fields__oSets__oPoints__observations__|'),
    ('observation',    'natural_enemy', 'number','fields__oSets__oPoints__observations__|__number'),
)
columns = pandas.MultiIndex.from_tuples(columns, names=names)
df = concat_df.reindex(
    columns=columns,
    level='source_field')
df.head()

scope                                      field                            \
group                                                                        
variable                                  client         crop         date   
source_field         fields__client__displayText fields__crop fields__date   
worksheet_name index                                                         
Erl            0                             NaN          NaN          NaN   
               1                             NaN          NaN          NaN   
               2                             NaN          NaN          NaN   
               3                             NaN          NaN          NaN   
               4                             NaN          NaN          NaN   

scope                                                               set  \
group                                                                     
variable                     desc         name                     date   
source_field         fields__desc fields__name      fields__oSets__date   
worksheet_name index                                                      
Erl            0              NaN          NaN  2017-08-02T13:12:09.542   
               1              NaN          NaN                      NaN   
               2              NaN          NaN                      NaN   
               3              NaN          NaN                      NaN   
               4              NaN          NaN                      NaN   

scope                                                                \
group                                                                 
variable                     description               growth_stage   
source_field         fields__oSets__desc fields__oSets__growthStage   
worksheet_name index                                                  
Erl            0                     NaN                        7.0   
               1                     NaN                        NaN   
               2                     NaN                        NaN   
               3                     NaN                        NaN   
               4                     NaN                        NaN   

scope                                                                    \
group                                                                     
variable                           observer               complete_sets   
source_field         fields__oSets__obsName fields__oSets__completeSets   
worksheet_name index                                                      
Erl            0                      Tyler                         0.0   
               1                        NaN                         NaN   
               2                        NaN                         NaN   
               3                        NaN                         NaN   
               4                        NaN                         NaN   

scope                                      ...                        \
group                                      ...                         
variable                                   ...                         
source_field                               ...                         
worksheet_name index                       ...                         
Erl            0                           ...                         
               1                           ...                         
               2                           ...                         
               3                           ...                         
               4                           ...                         

scope                                             observation  \
group                                                           
variable                                                   id   
source_field         fields__oSets__oPoints__observations__id   
worksheet_name index     

In [174]:
df.columns = df.columns.droplevel(level='source_field')
df.head()

scope                 field                                          set  \
group                                                                      
variable             client crop date desc name                     date   
worksheet_name index                                                       
Erl            0        NaN  NaN  NaN  NaN  NaN  2017-08-02T13:12:09.542   
               1        NaN  NaN  NaN  NaN  NaN                      NaN   
               2        NaN  NaN  NaN  NaN  NaN                      NaN   
               3        NaN  NaN  NaN  NaN  NaN                      NaN   
               4        NaN  NaN  NaN  NaN  NaN                      NaN   

scope                                                                 ...    \
group                                                                 ...     
variable             description growth_stage observer complete_sets  ...     
worksheet_name index                                                  ...     
Erl            0             NaN          7.0    Tyler           0.0  ...     
               1             NaN          NaN      NaN           NaN  ...     
               2             NaN          NaN      NaN           NaN  ...     
               3             NaN          NaN      NaN           NaN  ...     
               4             NaN          NaN      NaN           NaN  ...     

scope                observation                                           \
group                                                                       
variable                      id complete disabled natural_enemy_subtotal   
worksheet_name index                                                        
Erl            0             0.0      NaN      NaN                    NaN   
               1             1.0      NaN      NaN                    NaN   
               2             2.0      NaN      NaN                    NaN   
               3             3.0      NaN      NaN                    NaN   
               4             4.0      NaN      NaN                    NaN   

scope                                                                 
group                                           natural_enemy         
variable             aphid_subtotal  a1  a2  a3          name number  
worksheet_name index                                                  
Erl            0                0.0 NaN NaN NaN           NaN    NaN  
               1                0.0 NaN NaN NaN           NaN    NaN  
               2                0.0 NaN NaN NaN           NaN    NaN  
               3                0.0 NaN NaN NaN           NaN    NaN  
               4                0.0 NaN NaN NaN           NaN    NaN  

[5 rows x 22 columns]

## convert_datetime

In [234]:
date_column_mask, date_column_slice = df.columns.get_loc_level('date', level='variable')

In [235]:
df.loc[:, date_column_mask] = df.loc[:, date_column_mask].apply(pandas.to_datetime)
df.head()

scope                                      field                            \
group                                                                        
variable                                  client         crop         date   
source_field         fields__client__displayText fields__crop fields__date   
worksheet_name index                                                         
Erl            0                             NaN          NaN          NaT   
               1                             NaN          NaN          NaT   
               2                             NaN          NaN          NaT   
               3                             NaN          NaN          NaT   
               4                             NaN          NaN          NaT   

scope                                                              set  \
group                                                                    
variable                     desc         name                    date   
source_field         fields__desc fields__name     fields__oSets__date   
worksheet_name index                                                     
Erl            0              NaN          NaN 2017-08-02 13:12:09.542   
               1              NaN          NaN                     NaT   
               2              NaN          NaN                     NaT   
               3              NaN          NaN                     NaT   
               4              NaN          NaN                     NaT   

scope                                                                \
group                                                                 
variable                     description               growth_stage   
source_field         fields__oSets__desc fields__oSets__growthStage   
worksheet_name index                                                  
Erl            0                     NaN                        7.0   
               1                     NaN                        NaN   
               2                     NaN                        NaN   
               3                     NaN                        NaN   
               4                     NaN                        NaN   

scope                                                                    \
group                                                                     
variable                           observer               complete_sets   
source_field         fields__oSets__obsName fields__oSets__completeSets   
worksheet_name index                                                      
Erl            0                      Tyler                         0.0   
               1                        NaN                         NaN   
               2                        NaN                         NaN   
               3                        NaN                         NaN   
               4                        NaN                         NaN   

scope                                      ...                        \
group                                      ...                         
variable                                   ...                         
source_field                               ...                         
worksheet_name index                       ...                         
Erl            0                           ...                         
               1                           ...                         
               2                           ...                         
               3                           ...                         
               4                           ...                         

scope                                             observation  \
group                                                           
variable                                                   id   
source_field         fields__oSets__oPoints__observations__id   
worksheet_name index               

## forward fill

In [60]:
fill_columns = [
    ('point', 'id'),
    ('observation', 'id'),
] + list(df[['field',
             'set'
            ]].columns.values)
df.loc[:, fill_columns] = df.loc[:, fill_columns].ffill()

In [70]:
# df = (
#     df
#     .set_index([('set', 'date'), ('point', 'id'), ('observation', 'id'), ('natural_enemy', 'name')])
#     .sort_index()
#     .reset_index()
# )
# df.head(15)

## index

This index skips rows not containing an observation total from the source app (which should be the rows not needed after unstacking natural enemies).

In [71]:
index_columns = [('set', 'date'), ('point', 'id'), ('observation', 'id')]

In [77]:
index_df = (
    df.loc[
        df[
            [
                ('observation', 'aphid_subtotal'), 
                ('point', 'natural_enemy_subtotal'),
            ]
        ]
        .notna().any(axis='columns')
#         , []
    ]
)

index_df.head(15)

scope                                                                                        point  \
variable                                                                    natural_enemy_subtotal   
(set, date)             (point, id) (observation, id) (natural_enemy, name)                          
2017-07-14 12:31:24.194 0.0         0.0               NaN                                      NaN   
                                    1.0               NaN                                      NaN   
                                    2.0               NaN                                      NaN   
                                    3.0               NaN                                      NaN   
                                    4.0               NaN                                      NaN   
                                    5.0               e1                                       2.0   
                        1.0         0.0               NaN                                      NaN   
                                    1.0               NaN                                      NaN   
                                    2.0               NaN                                      NaN   
                                    3.0               NaN                                      NaN   
                                    4.0               NaN                                      NaN   
                                    5.0               e1                                       0.0   
                        2.0         0.0               NaN                                      NaN   
                                    1.0               NaN                                      NaN   
                                    2.0               NaN                                      NaN   

scope                                                                       natural_enemy  \
variable                                                                           number   
(set, date)             (point, id) (observation, id) (natural_enemy, name)                 
2017-07-14 12:31:24.194 0.0         0.0               NaN                             NaN   
                                    1.0               NaN                             NaN   
                                    2.0               NaN                             NaN   
                                    3.0               NaN                             NaN   
                                    4.0               NaN                             NaN   
                                    5.0               e1                              1.0   
                        1.0         0.0               NaN                             NaN   
                                    1.0               NaN                             NaN   
                                    2.0               NaN                             NaN   
                                    3.0               NaN                             NaN   
                                    4.0               NaN                             NaN   
                                    5.0               e1                              NaN   
                        2.0         0.0               NaN                             NaN   
                                    1.0               NaN                             NaN   
                                    2.0               NaN                             NaN   

scope                                                                          observation  \
variable                                                                    aphid_subtotal   
(set, date)             (point, id) (observation, id) (natural_enemy, name)                  
2017-07-14 12:31:24.194 0.0         0.0               NaN                              0.0   
                                    1.0               NaN                              0.0   
                  

## split frame into parts

### aphid

In [79]:
aphid_columns = list(df[[
    ('observation', 'complete'),
    ('observation', 'disabled'),
]]) + list(df[[
    'aphid_number'
]].columns)
aphid_df = df[aphid_columns].reset_index(level=[('observation', 'id')])
aphid_df = (
    aphid_df.loc[aphid_df[('observation', 'id')] < 5]
    .fillna(0)
    .set_index([('observation', 'id')], append=True)
)
aphid_df.head(17)

scope                                                                       observation  \
variable                                                                       complete   
(set, date)             (point, id) (natural_enemy, name) (observation, id)               
2017-07-14 12:31:24.194 0.0         NaN                   0.0                       0.0   
                                                          1.0                       0.0   
                                                          2.0                       0.0   
                                                          3.0                       0.0   
                                                          4.0                       0.0   
                        1.0         NaN                   0.0                       0.0   
                                                          1.0                       0.0   
                                                          2.0                       0.0   
                                                          3.0                       0.0   
                                                          4.0                       0.0   
                        2.0         NaN                   0.0                       0.0   
                                                          1.0                       0.0   
                                                          2.0                       0.0   
                                                          3.0                       0.0   
                                                          4.0                       0.0   
                        3.0         NaN                   0.0                       0.0   
                                                          1.0                       0.0   

scope                                                                                 \
variable                                                                    disabled   
(set, date)             (point, id) (natural_enemy, name) (observation, id)            
2017-07-14 12:31:24.194 0.0         NaN                   0.0                    0.0   
                                                          1.0                    0.0   
                                                          2.0                    0.0   
                                                          3.0                    0.0   
                                                          4.0                    0.0   
                        1.0         NaN                   0.0                    0.0   
                                                          1.0                    0.0   
                                                          2.0                    0.0   
                                                          3.0                    0.0   
                                                          4.0                    0.0   
                        2.0         NaN                   0.0                    0.0   
                                                          1.0                    0.0   
                                                          2.0                    0.0   
                                                          3.0                    0.0   
                                                          4.0                    0.0   
                        3.0         NaN                   0.0                    0.0   
                                                          1.0                    0.0   

scope                                                                        aphid_number  \
variable                                                                    English grain   
(set, date)             (point, id) (natural_enemy, name) (observation, id)                 
2017-07-14 12:31:24.194 0.0         NaN                   0.0                         7.0   
                                              

### natural enemies

In [80]:
ne_columns = list(df[[('observation', 'complete'), ('observation', 'disabled')]].columns) + list(df[['natural_enemy']].columns)
ne = df.loc[:, ne_columns]
ne = ne.dropna(subset=df[[('natural_enemy', 'name')]].columns)
ne = ne.reset_index()
ne = ne.set_index([('set', 'date'), ('point', 'id'), ('observation', 'id'), ('natural_enemy', 'name')]).fillna(0)
ne.head(11)

KeyError: "[('natural_enemy', 'name')] not in index"

#### natural enemy unstack (transpose)

In [18]:
ne2 = ne.unstack()#.reset_index()
ne2[('observation', 'complete', '')] = ne2.pop(('observation', 'complete','e1'))
ne2[('observation', 'disabled', '')] = ne2.pop(('observation', 'disabled','e1'))
ne2.head()

scope                                                 observation            \
variable                                                 complete             
('natural_enemy', 'name')                                      e2   e3   e4   
(set, date)             (point, id) (observation, id)                         
2017-07-14 12:31:24.194 0.0         5.0                       0.0  0.0  0.0   
                        1.0         5.0                       0.0  0.0  0.0   
                        2.0         5.0                       0.0  0.0  0.0   
                        3.0         5.0                       0.0  0.0  0.0   
                        4.0         5.0                       0.0  0.0  0.0   

scope                                                                      \
variable                                                                    
('natural_enemy', 'name')                               e5   e6   e7   e8   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5.0                0.0  0.0  0.0  0.0   
                        1.0         5.0                0.0  0.0  0.0  0.0   
                        2.0         5.0                0.0  0.0  0.0  0.0   
                        3.0         5.0                0.0  0.0  0.0  0.0   
                        4.0         5.0                0.0  0.0  0.0  0.0   

scope                                                                     \
variable                                                   disabled        
('natural_enemy', 'name')                               e9       e2   e3   
(set, date)             (point, id) (observation, id)                      
2017-07-14 12:31:24.194 0.0         5.0                0.0      0.0  0.0   
                        1.0         5.0                0.0      0.0  0.0   
                        2.0         5.0                0.0      0.0  0.0   
                        3.0         5.0                0.0      0.0  0.0   
                        4.0         5.0                0.0      0.0  0.0   

scope                                                   ...    natural_enemy  \
variable                                                ...           number   
('natural_enemy', 'name')                               ...               e2   
(set, date)             (point, id) (observation, id)   ...                    
2017-07-14 12:31:24.194 0.0         5.0                 ...              1.0   
                        1.0         5.0                 ...              0.0   
                        2.0         5.0                 ...              0.0   
                        3.0         5.0                 ...              0.0   
                        4.0         5.0                 ...              0.0   

scope                                                                      \
variable                                                                    
('natural_enemy', 'name')                               e3   e4   e5   e6   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5.0                0.0  0.0  0.0  0.0   
                        1.0         5.0                0.0  0.0  0.0  0.0   
                        2.0         5.0                0.0  0.0  0.0  0.0   
                        3.0         5.0                0.0  0.0  0.0  0.0   
                        4.0         5.0                0.0  0.0  0.0  0.0   

scope                                                                 \
variable                                                               
('natural_enemy', 'name')                               e7   e8   e9   
(set, date)             (point, id) (observation, id)                  
2017-07-14 12:31:24.194 0.0         5.0                0.0  0.0  0.0   
                        1.0         5.0                0.0  0.0  0.0   
                        2.0         5.0                0.0  0.

In [19]:
# ne2 = ne2.set_index([('set', 'date', ''), ('point', 'id', ''), ('observation', 'id', '')])
natural_enemy_columns = list(ne2[['natural_enemy']].columns.values)
columns = [
    ('observation', 'complete', ''),
    ('observation', 'disabled', '')
] + natural_enemy_columns
ne2 = ne2[columns]
# ne2.index = ne2.index.set_names([('set', 'date', ''), ('point', 'id', ''), ('observation', 'id', '')])
ne2.head(11)

scope                                                 observation           \
variable                                                 complete disabled   
('natural_enemy', 'name')                                                    
(set, date)             (point, id) (observation, id)                        
2017-07-14 12:31:24.194 0.0         5.0                       1.0      0.0   
                        1.0         5.0                       1.0      0.0   
                        2.0         5.0                       1.0      0.0   
                        3.0         5.0                       1.0      0.0   
                        4.0         5.0                       1.0      0.0   
2017-07-18 10:31:22.263 0.0         5.0                       1.0      0.0   
                        1.0         5.0                       1.0      0.0   
                        2.0         5.0                       1.0      0.0   
                        3.0         5.0                       1.0      0.0   
                        4.0         5.0                       1.0      0.0   
2017-07-18 10:51:30.195 0.0         5.0                       1.0      0.0   

scope                                                 natural_enemy            \
variable                                                     number             
('natural_enemy', 'name')                                        e1   e2   e3   
(set, date)             (point, id) (observation, id)                           
2017-07-14 12:31:24.194 0.0         5.0                         1.0  1.0  0.0   
                        1.0         5.0                         0.0  0.0  0.0   
                        2.0         5.0                         0.0  0.0  0.0   
                        3.0         5.0                         0.0  0.0  0.0   
                        4.0         5.0                         0.0  0.0  0.0   
2017-07-18 10:31:22.263 0.0         5.0                         0.0  0.0  0.0   
                        1.0         5.0                         0.0  0.0  0.0   
                        2.0         5.0                         0.0  0.0  0.0   
                        3.0         5.0                         0.0  0.0  0.0   
                        4.0         5.0                         0.0  0.0  0.0   
2017-07-18 10:51:30.195 0.0         5.0                         0.0  0.0  0.0   

scope                                                                      \
variable                                                                    
('natural_enemy', 'name')                               e4   e5   e6   e7   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5.0                0.0  0.0  0.0  0.0   
                        1.0         5.0                0.0  0.0  0.0  0.0   
                        2.0         5.0                0.0  0.0  0.0  0.0   
                        3.0         5.0                0.0  0.0  0.0  0.0   
                        4.0         5.0                0.0  0.0  0.0  0.0   
2017-07-18 10:31:22.263 0.0         5.0                0.0  0.0  0.0  0.0   
                        1.0         5.0                0.0  0.0  0.0  0.0   
                        2.0         5.0                0.0  0.0  0.0  0.0   
                        3.0         5.0                0.0  0.0  0.0  0.0   
                        4.0         5.0                0.0  0.0  0.0  0.0   
2017-07-18 10:51:30.195 0.0         5.0                0.0  0.0  1.0  0.0   

scope                                                            
variable                                                         
('natural_enemy', 'name')                               e8   e9  
(set, date)             (point, id) (observation, id)            
2017-07-14 12:31:24.194 0.0         5.0                0.0  0.0  
                        1.0         5.0                0.0  0.0  
                        2.0         5.0                0.0  0

#### flatten unstacked column names

In [20]:
new_names2 = [tuple([c[0], c[2] or c[1], None]) for c in ne2.columns.values]
ne2.columns = pandas.MultiIndex.from_tuples(new_names2).droplevel(2)
ne2.head(7)

observation           \
                                                         complete disabled   
(set, date)             (point, id) (observation, id)                        
2017-07-14 12:31:24.194 0.0         5.0                       1.0      0.0   
                        1.0         5.0                       1.0      0.0   
                        2.0         5.0                       1.0      0.0   
                        3.0         5.0                       1.0      0.0   
                        4.0         5.0                       1.0      0.0   
2017-07-18 10:31:22.263 0.0         5.0                       1.0      0.0   
                        1.0         5.0                       1.0      0.0   

                                                      natural_enemy            \
                                                                 e1   e2   e3   
(set, date)             (point, id) (observation, id)                           
2017-07-14 12:31:24.194 0.0         5.0                         1.0  1.0  0.0   
                        1.0         5.0                         0.0  0.0  0.0   
                        2.0         5.0                         0.0  0.0  0.0   
                        3.0         5.0                         0.0  0.0  0.0   
                        4.0         5.0                         0.0  0.0  0.0   
2017-07-18 10:31:22.263 0.0         5.0                         0.0  0.0  0.0   
                        1.0         5.0                         0.0  0.0  0.0   

                                                                           \
                                                        e4   e5   e6   e7   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5.0                0.0  0.0  0.0  0.0   
                        1.0         5.0                0.0  0.0  0.0  0.0   
                        2.0         5.0                0.0  0.0  0.0  0.0   
                        3.0         5.0                0.0  0.0  0.0  0.0   
                        4.0         5.0                0.0  0.0  0.0  0.0   
2017-07-18 10:31:22.263 0.0         5.0                0.0  0.0  0.0  0.0   
                        1.0         5.0                0.0  0.0  0.0  0.0   

                                                                 
                                                        e8   e9  
(set, date)             (point, id) (observation, id)            
2017-07-14 12:31:24.194 0.0         5.0                0.0  0.0  
                        1.0         5.0                0.0  0.0  
                        2.0         5.0                0.0  0.0  
                        3.0         5.0                0.0  0.0  
                        4.0         5.0                0.0  0.0  
2017-07-18 10:31:22.263 0.0         5.0                0.0  0.0  
                        1.0         5.0                0.0  0.0

#### rename natural enemy columns

In [21]:
ne2 = ne2.rename(columns=natural_enemy_name, level=1)
ne2.head()

observation           \
                                                         complete disabled   
(set, date)             (point, id) (observation, id)                        
2017-07-14 12:31:24.194 0.0         5.0                       1.0      0.0   
                        1.0         5.0                       1.0      0.0   
                        2.0         5.0                       1.0      0.0   
                        3.0         5.0                       1.0      0.0   
                        4.0         5.0                       1.0      0.0   

                                                            natural_enemy  \
                                                      aphid_mummies_brown   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5.0                               1.0   
                        1.0         5.0                               0.0   
                        2.0         5.0                               0.0   
                        3.0         5.0                               0.0   
                        4.0         5.0                               0.0   

                                                                         \
                                                      aphid_mummies_blk   
(set, date)             (point, id) (observation, id)                     
2017-07-14 12:31:24.194 0.0         5.0                             1.0   
                        1.0         5.0                             0.0   
                        2.0         5.0                             0.0   
                        3.0         5.0                             0.0   
                        4.0         5.0                             0.0   

                                                                         \
                                                      ladybeetle_larvae   
(set, date)             (point, id) (observation, id)                     
2017-07-14 12:31:24.194 0.0         5.0                             0.0   
                        1.0         5.0                             0.0   
                        2.0         5.0                             0.0   
                        3.0         5.0                             0.0   
                        4.0         5.0                             0.0   

                                                                        \
                                                      ladybeetle_adult   
(set, date)             (point, id) (observation, id)                    
2017-07-14 12:31:24.194 0.0         5.0                            0.0   
                        1.0         5.0                            0.0   
                        2.0         5.0                            0.0   
                        3.0         5.0                            0.0   
                        4.0         5.0                            0.0   

                                                                       \
                                                      lacewing_larvae   
(set, date)             (point, id) (observation, id)                   
2017-07-14 12:31:24.194 0.0         5.0                           0.0   
                        1.0         5.0                           0.0   
                        2.0         5.0                           0.0   
                        3.0         5.0                           0.0   
                        4.0         5.0                           0.0   

                                                                           \
                                                      lacewing_adult   e7   
(set, date)             (point, id) (observation, id)                       
2017-07-14 12:31:24.194 0.0         5.0                          0.0  0.0   
                        1.0         5.0                          0.0  0.0   
                        2.0         5.0        

### totals from source app

In [ ]:
totals_columns = [
    ('observation', 'aphid_subtotal'),
    ('point', 'natural_enemy_subtotal'),
    ('set', 'completeSets'),
    ('set', 'totalA1'),
    ('set', 'totalA2'),
    ('set', 'totalA3'),
    ('set', 'totalA4'),
    ('set', 'totalSets')
]

In [ ]:
totals_df = df.loc[
    df[
        [
            ('observation', 'aphid_subtotal'), 
            ('point', 'natural_enemy_subtotal'),
        ]
    ].notna().any(axis='columns'),
    totals_columns
].sort_index(axis='columns')
totals_df.head(15)

### crop conditions

In [ ]:
field_columns = df[['field']].columns.values
crop_conditions = (
    df[
        [('set', 'growthStage')] + list(field_columns)
    ].reset_index(level=[('point', 'id'), ('observation', 'id')], drop=True)
)
crop_conditions = crop_conditions.loc[~crop_conditions.index.duplicated()]
crop_conditions = (
    crop_conditions.set_index(
        [
            ('field', 'crop'),
            ('field', 'name'),
            ('field', 'desc'),
        ],
        append=True
    ).reorder_levels(
        [
            ('field', 'crop'),
            ('field', 'name'),
            ('field', 'desc'),
            ('set', 'date'),
        ],
        axis='index')
    .sort_index()
)
crop_conditions.head()

## Merge Parts

### aphid & natural enemy observations

In [ ]:
observation_base = (
    pandas.concat([aphid_df, ne2])
    .loc[:, list(aphid_df.columns.values) + list(ne2.columns.values)]
    .sort_index()
)
observation_base.head()

## Prepare DataFrames

### sum by site

In [23]:
sum_by_site = df[['field', 'set']]
sum_by_site

scope                                                                  field  \
variable                                                  client_displayText   
(set, date)             (point, id) (observation, id)                          
2017-07-14 12:31:24.194 0.0         0.0                AAFC SRDC, Tyler Wist   
                                    1.0                AAFC SRDC, Tyler Wist   
                                    2.0                AAFC SRDC, Tyler Wist   
                                    3.0                AAFC SRDC, Tyler Wist   
                                    4.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                        1.0         0.0                AAFC SRDC, Tyler Wist   
                                    1.0                AAFC SRDC, Tyler Wist   
                                    2.0                AAFC SRDC, Tyler Wist   
                                    3.0                AAFC SRDC, Tyler Wist   
                                    4.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                        2.0         0.0                AAFC SRDC, Tyler Wist   
                                    1.0                AAFC SRDC, Tyler Wist   
...                                                                      ...   
2017-08-25 13:05:14.062 7.0         5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                        8.0         0.0                AAFC SRDC, Tyler Wist   
                                    1.0                AAFC SRDC, Tyler Wist   
                                    2.0                AAFC SRDC, Tyler Wist   
                                    3.0                AAFC SRDC, Tyler Wist   
                                    4.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   
                                    5.0                AAFC SRDC, Tyler Wist   


### sum by set

In [ ]:
sum_by_set = (
    observations_base
    .sum(level=0)
#     .join(crop_conditions_indexed)
#     .reset_index()
)
sum_by_set.head(5)

### sum by point

In [ ]:
sum_by_point = (
    observations_base.sum(
        level=[
            ('set', 'date'),
            ('point', 'id'),
        ]
    )
#     .join( crop_conditions_indexed )
#     .reset_index()
)
sum_by_point.head(7)

### all cleaned data

In [ ]:
# columns = 
# list(df[[
#     ('set', 'date'),
#     ('observation', 'complete'),
#     ('observation', 'disabled'),
#     ('point', 'id'),
#     ('observation', 'id'),
# ]].columns) +
# list(df[].columns)
crop_conditions_indexed = df.loc[
    df[('observation', 'aphid_subtotal')].notna() | df[('point', 'natural_enemy_subtotal')].notna(),
    [
    'field',
    'set',
]
]#.set_index(index_columns)
crop_conditions_indexed.head(7)

In [ ]:
joined_df = (
    observation_base
    .join(crop_conditions_indexed)
#     .reset_index()
)
joined_df#.sample(7).sort_index()

### crop conditions

In [ ]:
crop_conditions_reindexed = crop_conditions_indexed.reset_index(
    level=[
        ('observation', 'id'),
        ('point', 'id'),
    ],
    drop=True
)
crop_conditions_reindexed = (
    crop_conditions_reindexed[~crop_conditions_reindexed.index.duplicated()]
    .set_index(
        [
            ('field', 'crop'),
            ('field', 'name'),
            ('field', 'desc'),
        ], append=True)
    .reorder_levels(
        [
            ('field', 'crop'),
            ('field', 'name'),
            ('field', 'desc'),
            ('set', 'date'),
        ], axis='index')
    .sort_index()
#     .reset_index()
)
crop_conditions_reindexed.head()

# save file

In [ ]:
sum_by_point.columns = sum_by_point.columns.map('_'.join)
joined_df.columns = joined_df.columns.map('_'.join)
with pandas.ExcelWriter('out_df.xlsx') as file_writer:
    sites_df.to_excel(
        file_writer,
        sheet_name='2017 - CAM - sites',
        freeze_panes=(1, 1),
        index_label='row'
    )
    sum_by_site.to_excel(
        file_writer,
        sheet_name='2017 - CAM - sum by site',
        freeze_panes=(1, 1),
        index_label='row'
    )
    sum_by_set.to_excel(
        file_writer,
        sheet_name='2017 - CAM - sum by time',
        freeze_panes=(1, 1),
        index_label='row'
    )
    sum_by_point.to_excel(
        file_writer,
        sheet_name='2017 - CAM - sum by point',
        freeze_panes=(1, 1),
        index_label='row'
    )
    joined_df.to_excel(
        file_writer,
        sheet_name='2017 - CAM - all observations',
        freeze_panes=(1, 1),
        index_label='row'
    )
    file_writer.save()

In [ ]:
# @todo: insert enemy and aphid names